In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import spacy
import seaborn as sns
import nltk
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
!python -m spacy download el_core_news_sm

2023-07-16 11:19:54.965484: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 76.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('el_core_news_sm')


In [3]:
nlp = spacy.load('el_core_news_sm')

In [4]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), min_df=0.01, max_df=0.95)

In [5]:
start_page = 1
end_page = 4

start_url = 'https://www.kathimerini.gr/search/%CE%B1%CE%BD%CF%84%CE%B9%CE%B5%CE%BC%CE%B2%CE%BF%CE%BB%CE%B9%CE%B1%CF%83%CF%84%CE%AD%CF%82/page/'

test_url = start_url + str(start_page)

test_url

'https://www.kathimerini.gr/search/%CE%B1%CE%BD%CF%84%CE%B9%CE%B5%CE%BC%CE%B2%CE%BF%CE%BB%CE%B9%CE%B1%CF%83%CF%84%CE%AD%CF%82/page/1'

In [6]:
response = requests.get(test_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [7]:
teaser_articles_list = doc.find('div', {'class':'columns is-flex-wrap-wrap'}).find_all('div', {'class': 'nx-article px-0 column is-full cat-template3'})

In [8]:
teaser_articles_list

[<div class="nx-article px-0 column is-full cat-template3">
 <div class="card columns is-flex-direction-row cat-list-rtl m-0" id="post-562438783">
 <div class="card-date column py-5 px-0">
 25.05.2023 </div>
 <div class="card-content column">
 <div class="media mb-4">
 <div class="media-content">
 <p class="subtitle is-7 mb-3">ΚΟΜΜΑΤΑ</p>
 <a href="https://www.kathimerini.gr/politics/562438783/ekloges-2023-i-niki-oi-mones-kai-oi-antiemvoliastes/">
 <p class="title is-5 medium_title">Εκλογές 2023: Η Νίκη, οι μονές και οι αντιεμβολιαστές</p>
 </a>
 </div>
 </div>
 <div class="content is-5 medium_lead is-hidden-touch">
 <a href="https://www.kathimerini.gr/politics/562438783/ekloges-2023-i-niki-oi-mones-kai-oi-antiemvoliastes/">
 <p>Οι ενδείξεις που συνδέουν το κόμμα-έκπληξη των εκλογών με παραεκκλησιαστικές οργανώσεις και αμφιλεγόμενα κινήματα.</p>
 </a>
 </div>
 </div>
 <div class="card-image column">
 <a class="is-elem-fullheight" href="https://www.kathimerini.gr/politics/562438783/eklo

In [9]:
link = teaser_articles_list[0].find('a', {'class': 'is-elem-fullheight'})['href']

link

'https://www.kathimerini.gr/politics/562438783/ekloges-2023-i-niki-oi-mones-kai-oi-antiemvoliastes/'

In [10]:
teasers_list = []

for page_num in range(start_page, end_page+1):
  page_url = start_url + str(page_num)
  response = requests.get(page_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo teaser_articles_list***
  teaser_articles_list = doc.find('div', {'class':'columns is-flex-wrap-wrap'}).find_all('div', {'class': 'nx-article px-0 column is-full cat-template3'})


  for teaser in teaser_articles_list:
      story_dict = {}
      try:

        #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στο link, ΜΕΤΑ το teaser
        link = teaser.find('a', {'class': 'is-elem-fullheight'})['href']


        if link.startswith("http"):
          story_dict['url'] = link
        else:
          story_dict['url'] = main_url + link
      except:
        story_dict['url'] = ''
        pass
      teasers_list.append(story_dict)
  time.sleep(1)

In [11]:
kathimerini_teasers_df = pd.DataFrame(teasers_list)
kathimerini_teasers_df

,url
0,https://www.kathimerini.gr/politics/562438783/...
1,https://www.kathimerini.gr/world/562405330/ita...
2,https://www.kathimerini.gr/world/562358347/ipa...
3,https://www.kathimerini.gr/society/562062805/a...
4,https://www.kathimerini.gr/world/562005436/ant...
5,https://www.kathimerini.gr/world/561714175/kan...
6,https://www.kathimerini.gr/world/561709321/ant...
7,https://www.kathimerini.gr/world/561708826/ita...
8,https://www.kathimerini.gr/world/561696361/par...
9,https://www.kathimerini.gr/world/561695830/tzo...


In [12]:
kathimerini_teasers_df.to_csv('/content/temp/kathimerini_teasers.csv', index=False)

In [13]:
article_url = 'https://www.kathimerini.gr/politics/561594844/g-oikonomoy-o-k-polakis-amfisvitei-toys-epistimones-kai-epimenei-na-stirizei-toys-antiemvoliastes/'

In [14]:
response = requests.get(article_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [15]:
article = doc.find('div', {'class': 'columns m-0 pt-2 is-flex-wrap-wrap'})

In [16]:
section = doc.find('span', {'class': 'nx-single-category-title color-primary'}).text
section

'Πολιτική'

In [17]:
title = article.find('h1', {'class': 'entry-title mb-5 mt-2'}).text
title

'Γ. Οικονόμου: Ο κ. Πολάκης αμφισβητεί τους επιστήμονες και επιμένει να στηρίζει τους αντιεμβολιαστές'

In [18]:
date = article.find('time', {'class': 'entry-date published'})['datetime']
date

'2021-11-19T22:30:07+02:00'

In [19]:
author = article.find('a', {'class', "url fn n"}).text
author

'\nNewsroom'

In [20]:
paragraphs = article.find_all('p')
paragraphs

[<p>«Την ώρα που σχεδόν 9 στους 10 διασωληνωμένους στις ΜΕΘ δεν έχουν κάνει το εμβόλιο, ο κ. Πολάκης αμφισβητεί τους επιστήμονες και επιμένει να στηρίζει τους αντιεμβολιαστές», τονίζει σε δήλωσή του ο κυβερνητικός εκπρόσωπος, <strong><a href="https://www.kathimerini.gr/tag/giannis-oikonomoy/">Γιάννης Οικονόμου,</a></strong> απαντώντας σε ανάρτηση του πρώην αναπληρωτή υπουργού Υγείας του ΣΥΡΙΖΑ.</p>,
 <p>Καλεί τον κ. Τσίπρα να τον αποδοκιμάσει δημόσια και υπογραμμίζει ότι «αν δεν το πράξει, θα επιβεβαιώσει για άλλη μια φορά ότι ο κ. Πολάκης είναι σε διατεταγμένη υπηρεσία». «Με σύμβουλο και συνεργάτη τον κ. Πολάκη, στον οποίο είχε εμπιστευθεί τη θέση του αναπληρωτή υπουργού Υγείας στην κυβέρνησή του, γίνεται αντιληπτό πως θα διαχειριζόταν την πανδημία» καταλήγει ο κ. Οικονόμου.</p>,
 <p><iframe allow="autoplay; clipboard-write; encrypted-media; picture-in-picture; web-share" allowfullscreen="allowfullscreen" class="lazy" data-src="https://www.facebook.com/plugins/post.php?href=https%3A%2

In [21]:
p_texts_list = []                      # δημιουργία κενής λίστας για να αποθηκεύσουμε το text κάθε παραγράφου
paragraphs = article.find_all('p')     # λίστα με όλες τις παραγράφους
for p in paragraphs:                   # για κάθε μια από τις παραγράφους στη λίστα
  p_texts_list.append(p.text)          # πάρε το .text και πρόσθεσέ το στη λίστα με τα texts των παραγράφων
full_text = ' '.join(p_texts_list)     # συνένωση των texts των παραγράφων
full_text = "".join(full_text.splitlines())
full_text

'«Την ώρα που σχεδόν 9 στους 10 διασωληνωμένους στις ΜΕΘ δεν έχουν κάνει το εμβόλιο, ο κ. Πολάκης αμφισβητεί τους επιστήμονες και επιμένει να στηρίζει τους αντιεμβολιαστές», τονίζει σε δήλωσή του ο κυβερνητικός εκπρόσωπος, Γιάννης Οικονόμου, απαντώντας σε ανάρτηση του πρώην αναπληρωτή υπουργού Υγείας του ΣΥΡΙΖΑ. Καλεί τον κ. Τσίπρα να τον αποδοκιμάσει δημόσια και υπογραμμίζει ότι «αν δεν το πράξει, θα επιβεβαιώσει για άλλη μια φορά ότι ο κ. Πολάκης είναι σε διατεταγμένη υπηρεσία». «Με σύμβουλο και συνεργάτη τον κ. Πολάκη, στον οποίο είχε εμπιστευθεί τη θέση του αναπληρωτή υπουργού Υγείας στην κυβέρνησή του, γίνεται αντιληπτό πως θα διαχειριζόταν την πανδημία» καταλήγει ο κ. Οικονόμου.  '

In [22]:
full_articles_list = []

for article_url in kathimerini_teasers_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo article που βρήκατε παραπάνω***
  article = doc.find('div', {'class': 'columns m-0 pt-2 is-flex-wrap-wrap'})

  full_article_dict = {}

  # website
  full_article_dict['site'] = "kathimerini.gr"

  # url
  full_article_dict['url'] = article_url

  #section
  try:
    section = doc.find('span', {'class': 'nx-single-category-title color-primary'}).text
    full_article_dict['section'] = section
  except:
    full_article_dict['section'] = ''
    pass

  #title
  try:
    title = article.find('h1', {'class': 'entry-title mb-5 mt-2'}).text
    full_article_dict['title'] = title
  except:
    full_article_dict['title'] = ''
    pass

  # date
  try:
    date = article.find('time', {'class': 'entry-date published'})['datetime']
    full_article_dict['date'] = date
  except:
    try:
      date = article.find('time', {'class': 'entry-date published updated'})['datetime']
      full_article_dict['date'] = date
    except:
      full_article_dict['date'] = ''
      pass

  # author
  try:
    author = article.find('a', {'class', "url fn n"}).text
    full_article_dict['author'] = author
  except:
    try:
      author = article.find('span', {'class', "url fn n"}).text
      full_article_dict['author'] = author
    except:
      full_article_dict['author'] = ''
      pass

  # full_text
  try:
    p_texts_list = []
    paragraphs = article.find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  full_articles_list.append(full_article_dict)
  time.sleep(1)

In [ ]:
kathimerini_full_articles_df = pd.DataFrame(full_articles_list)
kathimerini_full_articles_df

In [27]:
kathimerini_full_articles_df['full_text'] = kathimerini_full_articles_df['full_text'].str.replace('Ακολουθήστε το kathimerini.gr στο Google News και μάθετε πρώτοι όλες τις ειδήσεις Δείτε όλες τις τελευταίες Ειδήσεις από την Ελλάδα και τον Κόσμο, στο kathimerini.gr', ' ')

<ipython-input-27-c32ab1dda663>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  kathimerini_full_articles_df['full_text'] = kathimerini_full_articles_df['full_text'].str.replace('Ακολουθήστε το kathimerini.gr στο Google News και μάθετε πρώτοι όλες τις ειδήσεις Δείτε όλες τις τελευταίες Ειδήσεις από την Ελλάδα και τον Κόσμο, στο kathimerini.gr', ' ')


In [29]:
kathimerini_full_articles_df.to_csv('/content/temp/kathimerini_full_articles.csv', index=False)